In [1]:
import os.path
import sqlite3
import socket
import numpy as np


from torch.utils.data import Dataset

if socket.gethostname() == 'LTSSL-sKTPpP5Xl':
    data_dir = 'C:\\Users\\ams90\\PycharmProjects\\ConceptsBirds\\data'
elif socket.gethostname() == 'LAPTOP-NA88OLS1':
    data_dir = 'D:\\data\\caltecBirds\\CUB_200_2011'
else:
    data_dir = '/home/bwc/ams90/datasets/caltecBirds/CUB_200_2011'


In [2]:
conn = sqlite3.connect(database=os.path.join(data_dir, 'birds.db'))
conn.create_function("log", 1, np.log)
conn.create_function("exp", 1, np.exp)
cursor = conn.cursor()

In [3]:
classes      = {class_id: class_name     for (class_id, class_name)     in cursor.execute("select class_id, class_name from classes").fetchall()}
concepts     = {concept_id: concept_name for (concept_id, concept_name) in cursor.execute("select concept_id,concept_name from concepts").fetchall()}
attributes   = {attribute_id: attribute_id for (attribute_id, ) in cursor.execute("select attribute_id from attributes").fetchall()}
class_counts = {class_id: prior for (class_id, prior) in cursor.execute("select class_id, class_prior from nb_class_counts where trainset = 1").fetchall()}
class_counts

{1: 0.005005005005005005,
 2: 0.005005005005005005,
 3: 0.005005005005005005,
 4: 0.005005005005005005,
 5: 0.005005005005005005,
 6: 0.005005005005005005,
 7: 0.005005005005005005,
 8: 0.005005005005005005,
 9: 0.005005005005005005,
 10: 0.005005005005005005,
 11: 0.005005005005005005,
 12: 0.005005005005005005,
 13: 0.005005005005005005,
 14: 0.005005005005005005,
 15: 0.005005005005005005,
 16: 0.005005005005005005,
 17: 0.005005005005005005,
 18: 0.005005005005005005,
 19: 0.005005005005005005,
 20: 0.005005005005005005,
 21: 0.005005005005005005,
 22: 0.005005005005005005,
 23: 0.005005005005005005,
 24: 0.005005005005005005,
 25: 0.005005005005005005,
 26: 0.005005005005005005,
 27: 0.005005005005005005,
 28: 0.005005005005005005,
 29: 0.005005005005005005,
 30: 0.005005005005005005,
 31: 0.005005005005005005,
 32: 0.005005005005005005,
 33: 0.005005005005005005,
 34: 0.005005005005005005,
 35: 0.005005005005005005,
 36: 0.005005005005005005,
 37: 0.005005005005005005,
 38: 0.005

In [37]:
count, correct, unknown, wrong = 0, 0, 0, 0
loop = conn.cursor()

count, correct = 0, [0,0,0]
for image_id, class_id in loop.execute("select image_id, class_id from images where trainset = 1").fetchall():
    
    present_attribute_ids = [row[0] for row in cursor.execute("select attribute_id from image_attributes where present = 1 and image_id = ?", (image_id,)).fetchall()]
    
    class_log_probabilities = [row for row in cursor.execute("""select  sum(p.naive_test_probability),
                                                                        sum(p.naive_train_probability),
                                                                        sum(p.probability)
                                                                 from   class_attribute_probabilities p,
                                                                       image_attributes ia
                                                                 where  p.attribute_id = ia.attribute_id
                                                                 and    ia.present     = 1 
                                                                 and    ia.image_id    = ?
                                                                 group by p.class_id
                                                                 order by p.class_id """, (image_id,)).fetchall()]
    
    class_probabilities = np.exp(class_log_probabilities)
    class_probabilities = [[field for field in row] for row in class_probabilities]
    class_probabilities = [[row[column] for row in class_probabilities] for column in range(3)]
    

    for classifier in range(3):
    
        total_prob = sum(class_probabilities[classifier])       
        if total_prob > 0:
            norm_probabilities  = [probability / total_prob for probability in class_probabilities[classifier]]
            max_prob = max(norm_probabilities)
            pred_class_id = norm_probabilities.index(max_prob) + 1
            if pred_class_id == class_id: correct[classifier] += 1
       
    
    count += 1

    print(image_id, class_id, correct, [round(corr/count,4) for corr in correct])
    

loop.close()




2 1 [0, 0, 0] [0.0, 0.0, 0.0]
4 1 [1, 1, 1] [0.5, 0.5, 0.5]
5 1 [1, 2, 2] [0.3333, 0.6667, 0.6667]
7 1 [1, 2, 2] [0.25, 0.5, 0.5]
8 1 [2, 2, 2] [0.4, 0.4, 0.4]
9 1 [2, 2, 2] [0.3333, 0.3333, 0.3333]
11 1 [2, 2, 2] [0.2857, 0.2857, 0.2857]
15 1 [2, 2, 2] [0.25, 0.25, 0.25]
18 1 [3, 2, 2] [0.3333, 0.2222, 0.2222]
20 1 [3, 2, 3] [0.3, 0.2, 0.3]
21 1 [3, 2, 3] [0.2727, 0.1818, 0.2727]
23 1 [4, 2, 3] [0.3333, 0.1667, 0.25]
30 1 [4, 3, 4] [0.3077, 0.2308, 0.3077]
31 1 [4, 3, 4] [0.2857, 0.2143, 0.2857]
32 1 [4, 3, 4] [0.2667, 0.2, 0.2667]
35 1 [4, 3, 4] [0.25, 0.1875, 0.25]
36 1 [4, 3, 4] [0.2353, 0.1765, 0.2353]
37 1 [4, 3, 4] [0.2222, 0.1667, 0.2222]
40 1 [5, 4, 5] [0.2632, 0.2105, 0.2632]
41 1 [6, 5, 6] [0.3, 0.25, 0.3]
42 1 [6, 6, 6] [0.2857, 0.2857, 0.2857]
43 1 [7, 6, 6] [0.3182, 0.2727, 0.2727]
46 1 [7, 7, 7] [0.3043, 0.3043, 0.3043]
49 1 [8, 7, 7] [0.3333, 0.2917, 0.2917]
51 1 [9, 7, 7] [0.36, 0.28, 0.28]
53 1 [9, 8, 7] [0.3462, 0.3077, 0.2692]
55 1 [9, 8, 7] [0.3333, 0.2963, 0.2593]